In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image

/compuworks/anaconda3/envs/hspark/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def calculate_miou(image1, image2):
    intersection = np.logical_and(image1, image2).sum()
    union = np.logical_or(image1, image2).sum()
    miou = intersection / union
    return miou

In [3]:
def visualize_overlap(image1, image2, output_path):
    overlap = cv2.addWeighted(image1, 0.5, image2, 1, 0)
    plt.imshow(overlap, cmap='gray')
    plt.axis('off')
    plt.savefig(output_path)
    plt.close()

In [4]:
def resize_image(image_path):
    image = Image.open(image_path).convert('L')
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])
    image_tensor = transform(image)
    image_array = image_tensor.numpy().squeeze() * 255
    return image_array.astype(np.uint8)

In [5]:
def process_folders(inference_dir, target_dir, output_dir):
    miou = 0.0
    count = 0

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for folder_name in os.listdir(inference_dir):
        inference_folder_path = os.path.join(inference_dir, folder_name)
        target_folder_path = os.path.join(target_dir, folder_name)
        
        if os.path.isdir(inference_folder_path) and os.path.isdir(target_folder_path):
            output_folder_path = os.path.join(output_dir, folder_name)
            if not os.path.exists(output_folder_path):
                os.makedirs(output_folder_path)
                
            for image_name in os.listdir(inference_folder_path):
                if image_name.endswith('.jpg'):
                    inference_image_path = os.path.join(inference_folder_path, image_name)
                    target_image_path = os.path.join(target_folder_path, image_name)
                    
                    if os.path.exists(target_image_path):
                        inference_image = resize_image(inference_image_path)
                        target_image = resize_image(target_image_path)
                        
                        iou = calculate_miou(inference_image, target_image)
                        miou += iou
                        count += 1
                        # print(f"mIoU for {folder_name}/{image_name}: {miou:.4f}")
                        
                        output_image_path = os.path.join(output_folder_path, image_name)
                        visualize_overlap(inference_image, target_image, output_image_path)

    return miou / count

In [6]:
base_dir = 'hot'
inference_dir = os.path.join(base_dir, 'inference')
rule_inference_dir = os.path.join(base_dir, 'rule_inference')
target_dir = os.path.join(base_dir, 'target')

inference_output_dir = os.path.join(base_dir, 'inference_output')
rule_inference_output_dir = os.path.join(base_dir, 'rule_inference_output')

print("Deep Learning")
miou = process_folders(inference_dir, target_dir, inference_output_dir)
print(miou)
print("Rule Based")
rule_miou = process_folders(rule_inference_dir, target_dir, rule_inference_output_dir)
print(rule_miou)

Deep Learning
0.5538659474189714
Rule Based
0.6010612750121942
